# Pago Online: Scraping de URLs legítimas

**Fecha de ejecución:** 22/07/2025  
**Empresas objetivo:**  
PayPal, Mastercard, Visa, American Express, Bizum, Stripe, Revolut, Wise

---


| Empresa          | URLs únicas | Fecha      | Estado / Observaciones                          |
|------------------|------------|------------|-------------------------------------------------|
| Stripe           |    181     | 22/07/2025 | OK                                              |
| Wise             |    168     | 22/07/2025 | OK                                              |
| American Express |    117     | 22/07/2025 | OK                                              |
| PayPal           |     63     | 22/07/2025 | OK                                              |
| Bizum            |      1     | 22/07/2025 | OK – Muy pocos enlaces, home minimalista        |
| Revolut          |      1     | 22/07/2025 | OK – Muy pocos enlaces, home minimalista        |
| Mastercard       |             | 22/07/2025 | No ejecutado hoy / pendiente                    |
| Visa             |      0     | 22/07/2025 | Error SSL: CERTIFICATE_VERIFY_FAILED            |


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd, os
import time
from urllib.parse import urljoin, urlparse
from time import sleep


In [1]:
empresas_pago_online = {
    "PayPal": "https://www.paypal.com",
    "Mastercard": "https://www.mastercard.es",
    "Visa": "https://www.visa.es",
    "American Express": "https://www.americanexpress.com/es/",
    "Bizum": "https://bizum.es",
    "Stripe": "https://stripe.com/es",
    "Revolut": "https://www.revolut.com/es-ES",
    "Wise": "https://wise.com/es"
}


In [2]:
def obtener_urls(base_url, delay=1):
    """
Descarga la página principal y extrae las URLs internas
Acepta URLs del dominio principal y subdominios
"""
    try:
        # Realiza la petición HTTP a la URL de la enpresa con 10s máximos de espera
        response = requests.get(base_url, timeout=10)
        
        # Parsea el HTML recibido
        soup = BeautifulSoup(response.text, 'html.parser')

        # Obtiene el dominio principal
        dominio_empresa = urlparse(base_url).netloc

        # Elimina "www." al inicio para igualar subdominios y dominio principal

        if dominio_empresa.startswith('www.'):
            dominio_base = dominio_empresa[4:]
        else:
            dominio_base = dominio_empresa

        # Set para almacenar URLs únicas
        urls = set()

        # Itera por todos los enlaces (a, href)
        for link in soup.find_all('a', href=True):
            href = link['href'] # Extrae el href del enlace
            href = urljoin(base_url, href) #Convierte relativo a absoluto

            #Obtiene el dominio del enlace
            dominio_href = urlparse(href).netloc
            if dominio_href.startswith('www.'):
                dominio_href_base = dominio_href[4:]
            else:
                dominio_href_base = dominio_href

            # Añade la URL solo si es del dominio principal o subdomino
            if dominio_href_base == dominio_base or dominio_href_base.endswith('.' + dominio_base):
                urls.add(href)

        # Devuelve las URLs únicas como lista
        return list(urls)

    except Exception as e:
        print(f'Error al acceder a {base_url}: {e}')
        return []
 

        

        

    

In [5]:
resultados = []
for nombre, url_base in empresas_pago_online.items():
    print(f'Scraping empresa: {nombre}')
    urls_empresa = obtener_urls(url_base) 
    for url in urls_empresa:
        resultados.append({'empresa': nombre, 'url': url})
    time.sleep(2)
    

Scraping empresa: PayPal
Scraping empresa: Mastercard
Scraping empresa: Visa
Error al acceder a https://www.visa.es: HTTPSConnectionPool(host='www.visa.es', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1017)')))
Scraping empresa: American Express
Scraping empresa: Bizum
Scraping empresa: Stripe
Scraping empresa: Revolut
Scraping empresa: Wise


In [6]:
# Convertirmos los resultados en un DataFrame
df_scrap = pd.DataFrame(resultados)
#Guardamos el DataFrame en un archivo CSV
df_scrap.to_csv('../data/raw/saas_legitimas_crudo.csv', index=False)

In [10]:
print(df_scrap['empresa'].value_counts())


empresa
Stripe              181
Wise                168
American Express    117
PayPal               63
Bizum                 1
Revolut               1
Name: count, dtype: int64


- **Empresas scrapeadas:** 8
- **Total URLs obtenidas:** 531
- **Patrones detectados:**
    - Stripe y Wise tienen muchas páginas públicas (probablemente docs, ayuda, login, marketing...).
    - Bizum y Revolut solo muestran 1 URL válida, probablemente por home extremadamente minimalista o bloqueada.
- **Incidencias:**
    - Visa no permite scraping básico por error de certificado SSL (muy común en sitios protegidos).
    - Mastercard no ejecutado (completar en próxima ronda si es necesario).
- **Siguiente paso:**  
    - Decidir si necesitas intentar scraping avanzado en Visa/Mastercard o con lo obtenido te basta para la demo/modelo.
